In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
complete_df = pd.read_excel('data_okeanos.xlsx')

In [3]:
def opschonen(data):
    ### Onnodige kolommen verwijderen
    complete_df = pd.read_excel(data)
    clean_col_df = complete_df.drop(columns=['datum', 'ploeg', 'zone', 'intervaltype', 'interval_afstand', 'aantal_intervallen', 'interval_tijd', 'interval_nummer', 'rust', 'machine', 'spm'], errors='ignore')

    ### Lege strings omzetten naar NaN in de specifieke kolommen
    clean_col_df['500_split'] = clean_col_df['500_split'].replace('', pd.NA)
    clean_col_df['2k tijd'] = clean_col_df['2k tijd'].replace('', pd.NA)

    ### Alleen rijen met lege '500_split' of '2k tijd' verwijderen
    clean_row_df = clean_col_df.dropna(subset=['500_split', '2k tijd'])
    clean_row_df.dropna(how='any',inplace=True)

    ### Maak een kopie voor opschonen
    clean_df = clean_row_df.copy()
    clean_df.dropna(subset=['500_split', '2k tijd'])

    ### Komma's naar punten veranderen in ‘500_split’ en ‘2k tijd’ columns en uren veranderen naar minuten (00:01:11.11 wordt 1:11.11)
    for col in ['500_split', '2k tijd']:
        clean_df[col] = clean_df[col].astype(str).str.strip()
        clean_df[col] = clean_df[col].str.replace(',', '.', regex=False)
        clean_df[col] = clean_df[col].str.lstrip('0')

        # clean_df[col] = clean_df[col].astype(str).str.strip()
        # clean_df[col] = clean_df[col].str.replace(',', '.', regex=False)
        # clean_df[col] = clean_df[col].str.lstrip('0')
        # clean_df[col] = clean_df[col].str.replace('^0:0', '', regex=True)

    ### Verander 'trainingype' naar 'trainingtype'
    clean_df = clean_df.rename(columns={'trainingype': 'trainingtype'})
 
    return clean_df


### de minuten veranderen naar seconden

def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        minutes, seconds = map(float, str(time_str).split(':'))
        # print('GOOD:', time_str)
        return minutes * 60 + seconds
    except ValueError:
        # print('ERROR:', time_str)
        return None

def verander_omzetting_seconden(docu):
    docu['500_split'] = docu['500_split'].apply(time_to_seconds)
    docu['2k tijd'] = docu['2k tijd'].apply(time_to_seconds)
    return docu


In [4]:
roeien_opgeschoond = opschonen('data_okeanos.xlsx')
print(roeien_opgeschoond)

      ervaring geslacht gewichtsklasse   naam trainingtype      500_split  \
0          1.0        M              Z    z47         5x5'         1:44.6   
1          1.0        M              Z    z47         5x5'         1:44.7   
2          1.0        M              Z    z47         5x5'         1:44.3   
3          1.0        M              Z    z47         5x5'         1:44.0   
4          1.0        M              Z    z47         5x5'         1:44.1   
...        ...      ...            ...    ...          ...            ...   
7433       1.0        M              L  Z2522       1x2000  :06:37.400000   
7434       1.0        M              L  Z2521       1x2000  :06:57.500000   
7435       1.0        M              L  Z2520       1x2000         :07:05   
7436       1.0        M              Z  Z2517       1x2000         6:39.1   
7437       1.0        M              Z  Z2519       1x2000         6:17.0   

      2k tijd   2k datum  
0      6:19.9 2018-12-08  
1      6:19.9 2018-12

/tmp/ipykernel_27089/3565298680.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_row_df.dropna(how='any',inplace=True)


In [5]:
dataframe_op_sec = verander_omzetting_seconden(roeien_opgeschoond)
dataframe_op_sec.dropna(inplace=True)
print(dataframe_op_sec)

dataframe_op_sec.to_csv('verschoond_dataframe.csv', index=False)
verschoond_dataframe = pd.read_csv('verschoond_dataframe.csv')
verschoond_dataframe.to_excel('verschoond_dataframe.xlsx', index=None, header=True)

      ervaring geslacht gewichtsklasse   naam trainingtype  500_split  \
0          1.0        M              Z    z47         5x5'      104.6   
1          1.0        M              Z    z47         5x5'      104.7   
2          1.0        M              Z    z47         5x5'      104.3   
3          1.0        M              Z    z47         5x5'      104.0   
4          1.0        M              Z    z47         5x5'      104.1   
...        ...      ...            ...    ...          ...        ...   
7212       0.0        V              L  D2511       1x1500      120.4   
7419       1.0        M              Z  Z2522       2x2000      387.9   
7420       1.0        M              Z  Z2522       2x2000      406.4   
7436       1.0        M              Z  Z2517       1x2000      399.1   
7437       1.0        M              Z  Z2519       1x2000      377.0   

      2k tijd   2k datum  
0       379.9 2018-12-08  
1       379.9 2018-12-08  
2       379.9 2018-12-08  
3       379.9 2

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

def omzetten_binair(data):
    # Read the input data
    df = pd.read_excel(data)
    
    # Clean 'trainingtype' column
    df['trainingtype'] = df['trainingtype'].str.replace(r"/[a-zA-Z0-9']{3,}", "", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace("HOP+3x1'r", "57'", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace(" ", "", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace("r", "", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace(r"minuutjes", "1'", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace(r'\b(?:\d+x)+1\'', "1'", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace(r'(\d+x\d+)(?!m|\')$', r'\1m', regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace(r'\b(\d{4})(?!m|\')\b', r'\1m', regex=True)

    # Get unique training types and their decimal indices
    unique_trainingtypes = df['trainingtype'].unique()
    trainingtype_to_decimal = {trainingtype: i for i, trainingtype in enumerate(unique_trainingtypes)}
    trainingtype_mapping = {trainingtype: bin(i)[2:] for trainingtype, i in trainingtype_to_decimal.items()}

    # Replace training types in DataFrame with their binary equivalents
    df['binary_trainingtype'] = df['trainingtype'].map(trainingtype_mapping)

    # Convert 'geslacht' (gender) to binary using LabelBinarizer
    geslacht_binarizer = LabelBinarizer()
    geslacht_binarizer.fit(df['geslacht'])
    geslacht_mapping = {str(label): bin(i)[2:] for i, label in enumerate(geslacht_binarizer.classes_)}  # Convert to string
    df['binary_geslacht'] = df['geslacht'].map(geslacht_mapping)

    # Convert 'gewichtsklasse' (weight class) to binary using LabelBinarizer
    gewichtsklasse_binarizer = LabelBinarizer()
    gewichtsklasse_binarizer.fit(df['gewichtsklasse'])
    gewichtsklasse_mapping = {str(label): bin(i)[2:] for i, label in enumerate(gewichtsklasse_binarizer.classes_)}  # Convert to string
    df['binary_gewichtsklasse'] = df['gewichtsklasse'].map(gewichtsklasse_mapping)

    return df, trainingtype_mapping, geslacht_mapping, gewichtsklasse_mapping

# Example Usage
new_dataframe, trainingtype_mapping, geslacht_mapping, gewichtsklasse_mapping = omzetten_binair('verschoond_dataframe.xlsx')

# Print the updated DataFrame and mappings
print(new_dataframe.head())
print("Trainingtype Mapping:", trainingtype_mapping)
print("Geslacht Mapping:", geslacht_mapping)
print("Gewichtsklasse Mapping:", gewichtsklasse_mapping)

   ervaring geslacht gewichtsklasse naam trainingtype  500_split  2k tijd  \
0         1        M              Z  z47         5x5'      104.6    379.9   
1         1        M              Z  z47         5x5'      104.7    379.9   
2         1        M              Z  z47         5x5'      104.3    379.9   
3         1        M              Z  z47         5x5'      104.0    379.9   
4         1        M              Z  z47         5x5'      104.1    379.9   

     2k datum binary_trainingtype binary_geslacht binary_gewichtsklasse  
0  2018-12-08                   0               0                     1  
1  2018-12-08                   0               0                     1  
2  2018-12-08                   0               0                     1  
3  2018-12-08                   0               0                     1  
4  2018-12-08                   0               0                     1  
Trainingtype Mapping: {"5x5'": '0', "3x15'": '1', "3x20'": '10', '3x2000m': '11', '6000m': '1

In [7]:
### per persoon per training de gemiddelde 500_split berekenen

def gemid_split(data):
    data['year'] = pd.to_datetime(data['2k datum']).dt.year
    
    data['unique_naam'] = data['naam'] + '_' + data['year'].astype(str)

    gemid_data = (data.groupby(['unique_naam', 'binary_trainingtype'], as_index=False).agg({'ervaring': 'first', 'geslacht': 'first', 'gewichtsklasse': 'first', '500_split': 'mean', '2k tijd': 'first', 'binary_geslacht': 'first', 'binary_gewichtsklasse': 'first'}))

    return gemid_data

final_df = gemid_split(new_dataframe)
print(final_df.head())

# Save the result to a CSV file
final_df.to_csv('new_dataframe.csv', index=False)


  unique_naam binary_trainingtype  ervaring geslacht gewichtsklasse  \
0  D2510_2024               10000         0        V              Z   
1  D2510_2024                1001         0        V              Z   
2  D2510_2024              101100         0        V              Z   
3  D2510_2024                1100         0        V              Z   
4  D2511_2024               10000         0        V              L   

    500_split  2k tijd binary_geslacht binary_gewichtsklasse  
0  129.822222    496.3               1                     1  
1  113.638462    496.3               1                     1  
2  130.800000    496.3               1                     1  
3  125.116667    496.3               1                     1  
4  132.408333    484.1               1                     0  


In [8]:
### een dictionary maken van alle gegevens per persoon

import json

def dict_per_naam(data):
  naam_dict = {}

  for i in range(len(data)):
      row = data.iloc[i].to_dict()

      for key, value in row.items():
          if isinstance(value, pd.Timestamp):
              row[key] = value.strftime('%Y-%m-%d')

      name = row.pop('unique_naam')
      constants = {key: row.pop(key) for key in ['ervaring', 'geslacht', 'gewichtsklasse', '2k tijd']}

      if name not in naam_dict:
          naam_dict[name] = {'gegevens': constants, 'tijden': []}

      naam_dict[name]['tijden'].append(row)

  print(json.dumps(naam_dict, indent=4, sort_keys=False))

  return naam_dict

new_namen_dict = dict_per_naam(new_dataframe)
gemid_new_namen_dict = dict_per_naam(final_df)
print(len(gemid_new_namen_dict))

# groot_namen_dict = dict_per_naam(dataframe_op_sec)
# gemid_groot_namen_dict = dict_per_naam(groot_gemid_df)
# print(len(gemid_groot_namen_dict))

{
    "z47_2018": {
        "gegevens": {
            "ervaring": 1,
            "geslacht": "M",
            "gewichtsklasse": "Z",
            "2k tijd": 379.9
        },
        "tijden": [
            {
                "naam": "z47",
                "trainingtype": "5x5'",
                "500_split": 104.6,
                "2k datum": "2018-12-08",
                "binary_trainingtype": "0",
                "binary_geslacht": "0",
                "binary_gewichtsklasse": "1",
                "year": 2018
            },
            {
                "naam": "z47",
                "trainingtype": "5x5'",
                "500_split": 104.7,
                "2k datum": "2018-12-08",
                "binary_trainingtype": "0",
                "binary_geslacht": "0",
                "binary_gewichtsklasse": "1",
                "year": 2018
            },
            {
                "naam": "z47",
                "trainingtype": "5x5'",
                "500_split": 104.3,
            

In [9]:
from sklearn.model_selection import train_test_split

# # Data inladen
# df = groot_gemid_df

# # Eerst de data opschudden om bias te voorkomen
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Zorg ervoor dat elke ervaring, geslacht en gewichtsklasse in elke set vertegenwoordigd zijn
# train_data = pd.DataFrame()
# val_data = pd.DataFrame()
# test_data = pd.DataFrame()

# for ervaring in df['ervaring'].unique():
#     for geslacht in df['geslacht'].unique():
#         for gewichtsklasse in df['gewichtsklasse'].unique():
#             subset = df[(df['ervaring'] == ervaring) & (df['geslacht'] == geslacht) & (df['gewichtsklasse'] == gewichtsklasse)]
#             if not subset.empty:
#                 temp_train, temp_temp = train_test_split(subset, test_size=0.3, random_state=42)
#                 temp_val, temp_test = train_test_split(temp_temp, test_size=0.5, random_state=42)
#                 train_data = pd.concat([train_data, temp_train])
#                 val_data = pd.concat([val_data, temp_val])
#                 test_data = pd.concat([test_data, temp_test])

# # Reset indexen
# train_data.reset_index(drop=True, inplace=True)
# val_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)

# # Controleren op juiste verdeling
# print(f"Trainingsdata: {len(train_data)} rijen")
# print(f"Validatiedata: {len(val_data)} rijen")
# print(f"Testdata: {len(test_data)} rijen")

# # Optioneel: data opslaan in aparte bestanden
# train_data.to_csv('train_data.csv', index=False)
# val_data.to_csv('val_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)

In [10]:
def tweek_naar_split(dataframe_op_sec):
    for i in range(len(dataframe_op_sec['500_split'])):
        if dataframe_op_sec['500_split'].iloc[i] > 300:
          dataframe_op_sec['500_split'].iloc[i] = dataframe_op_sec['500_split'].iloc[i] / 4
    return dataframe_op_sec['500_split']

In [11]:
alles = tweek_naar_split(dataframe_op_sec)
print(alles)
print(len(alles))

min_value = alles.min()
filtered_alles = alles[alles!= min_value]

unique_values = alles.unique()
unique_values.sort()

first_min = unique_values[0]
second_min = unique_values[1]
third_min = unique_values[2]
fourth_min = unique_values[3]
eighth_min = unique_values[7]

print(first_min)
print(second_min)
print(third_min)
print(fourth_min)
print(eighth_min)

unique_values_desc = alles.unique()
unique_values_desc.sort()
unique_values_desc = unique_values_desc[::-1]

first_largest = unique_values_desc[0]
second_largest = unique_values_desc[1]
third_largest = unique_values_desc[2]
eighth_largest = unique_values_desc[7]

print(first_largest)
print(second_largest)
print(third_largest)
print(eighth_largest)

0       104.600
1       104.700
2       104.300
3       104.000
4       104.100
         ...   
7212    120.400
7419     96.975
7420    101.600
7436     99.775
7437     94.250
Name: 500_split, Length: 4316, dtype: float64
4316
64.0
64.7
81.7
83.7
90.0
165.7
159.0
158.1
151.3


/tmp/ipykernel_27089/816346744.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe_op_sec['500_split'].iloc[i] = dataframe_op_sec['500_split'].iloc[i] / 4
/tmp/ipykernel_27089/816346744.py:4: SettingWithCopyWarning: 
A value is try

In [12]:
def remove_outliers(dataframe_op_sec):
  alles = tweek_naar_split(dataframe_op_sec)
  alles = alles[alles >= 80]
  return alles

alles2 = remove_outliers(dataframe_op_sec)
print(len(alles2))

4314
